<a href="https://colab.research.google.com/github/Anthony-McGreevy/projectX/blob/master/Inventory_Stock_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1RgoONjxIOzYxZl36cE8V47dLjdp5yL72'
downloaded = drive.CreateFile({'id': file_id})

In [0]:
# Download the file to a local disk as 'exported.xlsx'.
downloaded.GetContentFile('online_retail(2009-2011).xlsx')

In [0]:
!pip install -q xlrd

import pandas as pd
df = pd.read_excel('online_retail(2009-2011).xlsx')
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1044843,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1044844,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1044845,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1044846,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


Things to do 

•Data sanity check and preprocessing 
- Use data visualisation techniques
- Missing Data
- Feature Selection techniques 
- Group data into hourly
- Add season (1,2,3,4)
- Add day of the week in separate column
- Deriving temperol features 
- Transformation using StandardScaler





In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [0]:
retail_df = pd.read_excel('online_retail(2009-2011).xlsx',usecols=['Invoice', 'StockCode', 'Quantity', 'InvoiceDate', 'Price'])

In [0]:
retail_df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price
0,489434,85048,12,2009-12-01 07:45:00,6.95
1,489434,79323P,12,2009-12-01 07:45:00,6.75
2,489434,79323W,12,2009-12-01 07:45:00,6.75
3,489434,22041,48,2009-12-01 07:45:00,2.10
4,489434,21232,24,2009-12-01 07:45:00,1.25


Only using the columns that will provide value to the model. 


In [0]:
for column in retail_df.columns:
    null_vals = retail_df[column].isnull().values.sum()
    if(null_vals!=0):
        print (column,null_vals)

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [0]:
retail_df.describe()

,Quantity,Price
count,1.044848e+06,1.044848e+06
mean,9.993649e+00,4.590546e+00
std,1.742185e+02,1.217042e+02
min,-8.099500e+04,-5.359436e+04
25%,1.000000e+00,1.250000e+00
50%,3.000000e+00,2.100000e+00
75%,1.000000e+01,4.130000e+00
max,8.099500e+04,3.897000e+04


Do we want to drop values that are below zero?

In [0]:
#dropping all value that have na or are duplications
retail_df.dropna(inplace=True)

retail_df.drop_duplicates(inplace=True)

retail_df.shape

(1033034, 5)

In [0]:
retail_df.StockCode = retail_df.StockCode.astype('category')

retail_df['InvoiceDate'] = pd.to_datetime(retail_df['InvoiceDate'], format = "%m/%d/%Y %H:%M")

retail_df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price
0,489434,85048,12,2009-12-01 07:45:00,6.95
1,489434,79323P,12,2009-12-01 07:45:00,6.75
2,489434,79323W,12,2009-12-01 07:45:00,6.75
3,489434,22041,48,2009-12-01 07:45:00,2.10
4,489434,21232,24,2009-12-01 07:45:00,1.25


In [0]:
min(retail_df.InvoiceDate), max(retail_df.InvoiceDate) 

(Timestamp('2009-12-01 07:45:00'), Timestamp('2011-12-09 12:50:00'))

Taking out any values that are below zero and any values that begin with a C in Invoice as that seems to be for products that have been cancelled.

In [0]:
retail_df['Invoice'] = retail_df['Invoice'].astype('str')

non_c_df = retail_df[retail_df["Invoice"].str.startswith('C')==False]

mask = (non_c_df["Quantity"]>0) & (non_c_df["Price"] >0)

grouped_hourly = non_c_df.loc[mask]

grouped_hourly.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price
0,489434,85048,12,2009-12-01 07:45:00,6.95
1,489434,79323P,12,2009-12-01 07:45:00,6.75
2,489434,79323W,12,2009-12-01 07:45:00,6.75
3,489434,22041,48,2009-12-01 07:45:00,2.10
4,489434,21232,24,2009-12-01 07:45:00,1.25


In [0]:
grouped_hourly.describe()

,Quantity,Price
count,1.007911e+06,1.007911e+06
mean,1.111690e+01,4.074253e+00
std,1.284698e+02,5.043050e+01
min,1.000000e+00,1.000000e-03
25%,1.000000e+00,1.250000e+00
50%,4.000000e+00,2.100000e+00
75%,1.200000e+01,4.130000e+00
max,8.099500e+04,2.511109e+04


In [0]:
#changing the time format into hourly as opposed to just minute 
from datetime import datetime
grouped_hourly['InvoiceDate'] = grouped_hourly['InvoiceDate'].apply(lambda x: pd.to_datetime(x).strftime("%y-%m-%d %H") ) 

In [0]:
grouped_hourly.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price
0,489434,85048,12,09-12-01 07,6.95
1,489434,79323P,12,09-12-01 07,6.75
2,489434,79323W,12,09-12-01 07,6.75
3,489434,22041,48,09-12-01 07,2.10
4,489434,21232,24,09-12-01 07,1.25


In [0]:
grouped_hourly_stock = grouped_hourly.groupby(['InvoiceDate','StockCode'])["Quantity"].sum().unstack()

In [0]:
grouped_hourly_stock.head()

StockCode,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,15002,15030,15034,15036,15039,16008,16010,16011,16012,16014,16015,16016,16033,16043,16044,16045,16046,16047,16048,16049,16050,16051,16052,16053,16054,16119,16131,16132,16192,16195,...,DCGS0057,DCGS0058,DCGS0059,DCGS0060,DCGS0062,DCGS0066N,DCGS0066P,DCGS0067,DCGS0068,DCGS0069,DCGS0070,DCGS0071,DCGS0072,DCGS0073,DCGS0074,DCGS0075,DCGS0076,DCGSLBOY,DCGSLGIRL,DCGSSBOY,DCGSSGIRL,DOT,GIFT,M,PADS,POST,S,SP1002,TEST001,TEST002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_60,gift_0001_70,gift_0001_80,gift_0001_90,m
InvoiceDate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
09-12-01 07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
09-12-01 09,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
09-12-01 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
09-12-01 11,NaN,NaN,NaN,NaN,NaN,6.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
09-12-01 12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
grouped_hourly_stock.fillna(0,inplace=True)

In [0]:
grouped_hourly_stock.columns = grouped_hourly_stock.columns.astype("object")

In [0]:
grouped_hourly_stock.reset_index(inplace=True)

In [0]:
grouped_hourly_stock.head()

StockCode,InvoiceDate,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,15002,15030,15034,15036,15039,16008,16010,16011,16012,16014,16015,16016,16033,16043,16044,16045,16046,16047,16048,16049,16050,16051,16052,16053,16054,16119,16131,16132,16192,...,DCGS0057,DCGS0058,DCGS0059,DCGS0060,DCGS0062,DCGS0066N,DCGS0066P,DCGS0067,DCGS0068,DCGS0069,DCGS0070,DCGS0071,DCGS0072,DCGS0073,DCGS0074,DCGS0075,DCGS0076,DCGSLBOY,DCGSLGIRL,DCGSSBOY,DCGSSGIRL,DOT,GIFT,M,PADS,POST,S,SP1002,TEST001,TEST002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_60,gift_0001_70,gift_0001_80,gift_0001_90,m
0,09-12-01 07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,09-12-01 09,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,09-12-01 10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,09-12-01 11,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,09-12-01 12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Deriving temporal features**

In [0]:
grouped_hourly_stock.InvoiceDate = pd.to_datetime(grouped_hourly_stock.InvoiceDate,format="%y-%m-%d %H")

In [0]:
grouped_hourly_stock['month'] = grouped_hourly_stock.InvoiceDate.dt.month
grouped_hourly_stock['Day_of_week'] = grouped_hourly_stock.InvoiceDate.dt.dayofweek

seasons = [1,1,2,2,2,3,3,3,4,4,4,1]
month_to_season = dict(zip(range(1,13), seasons))
season = grouped_hourly_stock.month.map(month_to_season)
grouped_hourly_stock['season'] = season

In [0]:
grouped_hourly_stock.head()

StockCode,InvoiceDate,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,15002,15030,15034,15036,15039,16008,16010,16011,16012,16014,16015,16016,16033,16043,16044,16045,16046,16047,16048,16049,16050,16051,16052,16053,16054,16119,16131,16132,16192,...,DCGS0060,DCGS0062,DCGS0066N,DCGS0066P,DCGS0067,DCGS0068,DCGS0069,DCGS0070,DCGS0071,DCGS0072,DCGS0073,DCGS0074,DCGS0075,DCGS0076,DCGSLBOY,DCGSLGIRL,DCGSSBOY,DCGSSGIRL,DOT,GIFT,M,PADS,POST,S,SP1002,TEST001,TEST002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_60,gift_0001_70,gift_0001_80,gift_0001_90,m,month,Day_of_week,season
0,2009-12-01 07:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1
1,2009-12-01 09:00:00,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1
2,2009-12-01 10:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1
3,2009-12-01 11:00:00,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1
4,2009-12-01 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1


Now we have added seasonality columns at the end: month, day of week, season


In [0]:
grouped_hourly_stock.shape

(5793, 5309)

I added a new column that will separate the days into morning, afternoon and night 

In [0]:
grouped_hourly_stock = grouped_hourly_stock.assign(session=pd.cut(grouped_hourly_stock.InvoiceDate.dt.hour,[0,6,12,16,20],labels=['Night','Morning','Afternoon','Evening']))

In [0]:
grouped_hourly_stock.head()

StockCode,InvoiceDate,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,15002,15030,15034,15036,15039,16008,16010,16011,16012,16014,16015,16016,16033,16043,16044,16045,16046,16047,16048,16049,16050,16051,16052,16053,16054,16119,16131,16132,16192,...,DCGS0062,DCGS0066N,DCGS0066P,DCGS0067,DCGS0068,DCGS0069,DCGS0070,DCGS0071,DCGS0072,DCGS0073,DCGS0074,DCGS0075,DCGS0076,DCGSLBOY,DCGSLGIRL,DCGSSBOY,DCGSSGIRL,DOT,GIFT,M,PADS,POST,S,SP1002,TEST001,TEST002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_60,gift_0001_70,gift_0001_80,gift_0001_90,m,month,Day_of_week,season,session
0,2009-12-01 07:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1,Morning
1,2009-12-01 09:00:00,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1,Morning
2,2009-12-01 10:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1,Morning
3,2009-12-01 11:00:00,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1,Morning
4,2009-12-01 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1,Morning


In [0]:
grouped_hourly_stock.drop(['InvoiceDate'],axis=1,inplace=True)

In [0]:
grouped_hourly_stock.columns

Index([         10002,          10080,          10109,          10120,
                10125,          10133,          10134,          10135,
                10138,          11001,
       ...
       'gift_0001_50', 'gift_0001_60', 'gift_0001_70', 'gift_0001_80',
       'gift_0001_90',            'm',        'month',  'Day_of_week',
             'season',      'session'],
      dtype='object', name='StockCode', length=5309)

In [0]:
category_cols = ['month', 'season', 'Day_of_week','session']

In [0]:
for column in category_cols:
    grouped_hourly_stock[column] = grouped_hourly_stock[column].astype("category")

In [0]:
expanded_trans_df = pd.get_dummies(grouped_hourly_stock)

In [0]:
expanded_trans_df.shape

(5793, 5332)

In [0]:
target_labels = 5304

In [0]:
expanded_trans_df.columns[:target_labels]

Index([         10002,          10080,          10109,          10120,
                10125,          10133,          10134,          10135,
                10138,          11001,
       ...
            'TEST002', 'gift_0001_10', 'gift_0001_20', 'gift_0001_30',
       'gift_0001_40', 'gift_0001_50', 'gift_0001_60', 'gift_0001_70',
       'gift_0001_80', 'gift_0001_90'],
      dtype='object', length=5304)

In [0]:
expanded_trans_df.head()

,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,15002,15030,15034,15036,15039,16008,16010,16011,16012,16014,16015,16016,16033,16043,16044,16045,16046,16047,16048,16049,16050,16051,16052,16053,16054,16119,16131,16132,16192,16195,...,SP1002,TEST001,TEST002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_60,gift_0001_70,gift_0001_80,gift_0001_90,m,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,Day_of_week_0,Day_of_week_1,Day_of_week_2,Day_of_week_3,Day_of_week_4,Day_of_week_5,Day_of_week_6,season_1,season_2,season_3,season_4,session_Night,session_Morning,session_Afternoon,session_Evening
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
1,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
3,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0


**Transformation using StandardScaler**

In [0]:
from sklearn.preprocessing import StandardScaler
expanded_scaler = StandardScaler(with_mean=False)     #to keep sparse matrix intact
expanded_scaled = expanded_scaler.fit_transform(expanded_trans_df)

In [0]:
# look back 24 timestamps back and 1 stamp ahead
n_hours = 24
n_features = expanded_trans_df.shape[1]

In [0]:
expanded_scaler.inverse_transform(expanded_scaled)

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [12.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

**LSTM Modelling**

In [0]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.utils import np_utils
from keras.optimizers import sgd,adam

Using TensorFlow backend.


In [0]:
#Referenced from : https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [0]:
reframed = series_to_supervised(expanded_scaled, n_hours,1)